In [168]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler
from sklearn.impute import SimpleImputer, KNNImputer
import seaborn as sns
import matplotlib.pyplot as plt

## Train Test Split

1. Pisahkan X (fitur prediktor) dan y (label) dari titanic  
2. Lakukan train test split (test size = 0.2, stratify = y) yang menghasilkan X_train, X_test, y_train, y_test  
3. Print .head(), type(), dan .shape dari masing-masing variabel  

In [169]:
titanic = pd.read_csv('titanic_new.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [170]:
X = titanic.drop(['Survived', 'PassengerId', 'Name', 'Parch', 'Ticket', 'Cabin'], axis=1)
y = titanic['Survived']

In [171]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state=42)

In [172]:
for set in (X_train, X_test, y_train, y_test):
    print(set.head())
    print(type(set))
    print(set.shape)
    print('---')

      Pclass     Sex   Age  SibSp      Fare Embarked
1021       3    male  32.0      0    8.0500        S
318        1  female  31.0      0  164.8667        S
924        3  female   NaN      1   23.4500        S
652        3    male  21.0      0    8.4333        S
1189       1    male  30.0      0   45.5000        S
<class 'pandas.core.frame.DataFrame'>
(1047, 6)
---
      Pclass     Sex   Age  SibSp      Fare Embarked
950        1  female  36.0      0  262.3750        C
811        3    male  39.0      0   24.1500        S
475        1    male   NaN      0   52.0000        S
1136       1    male  41.0      1   51.8625        S
137        1    male  37.0      1   53.1000        S
<class 'pandas.core.frame.DataFrame'>
(262, 6)
---
1021    0
318     1
924     1
652     0
1189    0
Name: Survived, dtype: int64
<class 'pandas.core.series.Series'>
(1047,)
---
950     1
811     0
475     0
1136    0
137     0
Name: Survived, dtype: int64
<class 'pandas.core.series.Series'>
(262,)
---


In [173]:
y_train.value_counts(normalize=True)

0    0.622732
1    0.377268
Name: Survived, dtype: float64

In [174]:
y_test.value_counts(normalize=True)

0    0.622137
1    0.377863
Name: Survived, dtype: float64

In [175]:
X_train.isna().sum()

Pclass        0
Sex           0
Age         204
SibSp         0
Fare          0
Embarked      0
dtype: int64

In [176]:
X_test.isna().sum()

Pclass       0
Sex          0
Age         59
SibSp        0
Fare         1
Embarked     2
dtype: int64

1. Lakukan knn imputation untuk handling missing value di X_train  
2. Buktikan bahwa sudah tidak ada missing value dengan menggunakan .isna().sum()  

In [177]:
sex_embarked = X_train[['Sex', 'Embarked']]

In [178]:
ordinal_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
sex_embarked_encoded = ordinal_enc.fit_transform(sex_embarked)

In [179]:
X_train_imp = X_train.copy(deep=True)

In [180]:
sex_embarked_encoded

array([[1., 2.],
       [0., 2.],
       [0., 2.],
       ...,
       [1., 2.],
       [0., 2.],
       [1., 2.]])

In [181]:
X_train_imp['Sex'] = sex_embarked_encoded[:,0]
X_train_imp['Embarked'] = sex_embarked_encoded[:,1]
X_train_imp.head()

,Pclass,Sex,Age,SibSp,Fare,Embarked
1021,3,1.0,32.0,0,8.0500,2.0
318,1,0.0,31.0,0,164.8667,2.0
924,3,0.0,NaN,1,23.4500,2.0
652,3,1.0,21.0,0,8.4333,2.0
1189,1,1.0,30.0,0,45.5000,2.0


In [182]:
knn_imp = KNNImputer(n_neighbors=5)

In [183]:
X_train_imp = pd.DataFrame(knn_imp.fit_transform(X_train_imp), columns=knn_imp.feature_names_in_)

In [184]:
X_train_imp.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Fare        0
Embarked    0
dtype: int64

In [185]:
X_train_imp.head()

,Pclass,Sex,Age,SibSp,Fare,Embarked
0,3.0,1.0,32.0,0.0,8.0500,2.0
1,1.0,0.0,31.0,0.0,164.8667,2.0
2,3.0,0.0,22.2,1.0,23.4500,2.0
3,3.0,1.0,21.0,0.0,8.4333,2.0
4,1.0,1.0,30.0,0.0,45.5000,2.0


In [186]:
sex_embarked_imp = X_train_imp[['Sex', 'Embarked']]

In [187]:
sex_embarked_imp['Sex'].unique()

array([1., 0.])

In [188]:
sex_embarked_imp['Embarked'].unique()

array([2., 1., 0.])

In [189]:
sex_embarked_inv = ordinal_enc.inverse_transform(sex_embarked_imp)

In [190]:
X_train_imp['Sex'] = sex_embarked_inv[:,0]
X_train_imp['Embarked'] = sex_embarked_inv[:,1]

In [191]:
X_train_imp.head()

,Pclass,Sex,Age,SibSp,Fare,Embarked
0,3.0,male,32.0,0.0,8.0500,S
1,1.0,female,31.0,0.0,164.8667,S
2,3.0,female,22.2,1.0,23.4500,S
3,3.0,male,21.0,0.0,8.4333,S
4,1.0,male,30.0,0.0,45.5000,S


## Feature Engineering

### Standard Scaling

In [192]:
dum = np.random.randint(1, 6, 10)
dum.shape

(10,)

In [193]:
dum2 = dum.reshape(-1, 1)
dum2.shape

(10, 1)

In [194]:
ss = StandardScaler()

In [195]:
dum2_ss = ss.fit_transform(dum2)

In [196]:
dum2_ss.mean()

4.4408920985006264e-17

In [197]:
dum2_ss.std()

1.0

### MinMax Scaling

In [198]:
mm = MinMaxScaler()

In [199]:
dum2_mm = mm.fit_transform(dum2)
dum2_mm

array([[0.  ],
       [0.5 ],
       [0.75],
       [0.  ],
       [0.25],
       [0.  ],
       [0.  ],
       [1.  ],
       [0.  ],
       [1.  ]])

In [200]:
dum2_mm.min()

0.0

In [201]:
dum2_mm.max()

1.0

### MaxAbs Scaling

In [202]:
maxabs = MaxAbsScaler()

In [203]:
dum2_maxabs = maxabs.fit_transform(dum2)
dum2_maxabs

array([[0.2],
       [0.6],
       [0.8],
       [0.2],
       [0.4],
       [0.2],
       [0.2],
       [1. ],
       [0.2],
       [1. ]])

In [204]:
np.max(np.abs(dum2))

5

In [205]:
dum2/np.max(np.abs(dum2))

array([[0.2],
       [0.6],
       [0.8],
       [0.2],
       [0.4],
       [0.2],
       [0.2],
       [1. ],
       [0.2],
       [1. ]])

### Robust Scaling

In [206]:
robust = RobustScaler()

In [207]:
dum2_robust = robust.fit_transform(dum2)
dum2_robust

array([[-0.18181818],
       [ 0.54545455],
       [ 0.90909091],
       [-0.18181818],
       [ 0.18181818],
       [-0.18181818],
       [-0.18181818],
       [ 1.27272727],
       [-0.18181818],
       [ 1.27272727]])

In [208]:
dum2

array([[1],
       [3],
       [4],
       [1],
       [2],
       [1],
       [1],
       [5],
       [1],
       [5]])

In [209]:
for n in [25, 50, 75]:
    print(np.percentile(dum2, n))

1.0
1.5
3.75


In [210]:
(dum2 - 3)/(3.75 - 2)

array([[-1.14285714],
       [ 0.        ],
       [ 0.57142857],
       [-1.14285714],
       [-0.57142857],
       [-1.14285714],
       [-1.14285714],
       [ 1.14285714],
       [-1.14285714],
       [ 1.14285714]])

1. Buat array random berisi 10 integer, print array  
2. Lakukan standar scaling, hitung mean dan std pra dan paska scaling, print array hasil scaling  
3. Lakukan min max scaling, hitung min dan max pra dan paska scaling, print array hasil scaling  
4. Lakukan max abs scaling, hitung max(|x|) yang akan dijadikan penyebut, print array hasil scaling  
5. Lakukan robust scaling, hitung q1, q2, q3 yang dijadikan parameter pada scaler, print array hasil scaling  

In [211]:
dum3 = np.random.randint(1, 8, 10)
dum3

array([4, 1, 2, 6, 6, 6, 4, 2, 5, 7])

In [212]:
dum4 = dum3.reshape(-1, 1)
dum4

array([[4],
       [1],
       [2],
       [6],
       [6],
       [6],
       [4],
       [2],
       [5],
       [7]])

In [213]:
dum4_ss = ss.fit_transform(dum4)
dum4_ss

array([[-0.15369466],
       [-1.69064121],
       [-1.1783257 ],
       [ 0.87093638],
       [ 0.87093638],
       [ 0.87093638],
       [-0.15369466],
       [-1.1783257 ],
       [ 0.35862086],
       [ 1.3832519 ]])

In [214]:
print('--Before Scaling--')
print('Mean: ', dum4.mean())
print('Std: ', dum4.std())
print('================')
print('--After Scaling--')
print('Mean: ', dum4_ss.mean())
print('Std: ', dum4_ss.std())

--Before Scaling--
Mean:  4.3
Std:  1.9519221295943134
--After Scaling--
Mean:  4.4408920985006264e-17
Std:  1.0


In [215]:
dum4_mm = mm.fit_transform(dum4)
dum4_mm

array([[0.5       ],
       [0.        ],
       [0.16666667],
       [0.83333333],
       [0.83333333],
       [0.83333333],
       [0.5       ],
       [0.16666667],
       [0.66666667],
       [1.        ]])

In [216]:
print('--Before Scaling--')
print('Min: ', dum4.max())
print('Min: ', dum4.min())
print('================')
print('--After Scaling--')
print('Min: ', dum4_mm.min())
print('Max: ', dum4_mm.max())

--Before Scaling--
Min:  7
Min:  1
--After Scaling--
Min:  0.0
Max:  0.9999999999999999


In [217]:
dum4_maxabs = maxabs.fit_transform(dum4)
dum4_maxabs

array([[0.57142857],
       [0.14285714],
       [0.28571429],
       [0.85714286],
       [0.85714286],
       [0.85714286],
       [0.57142857],
       [0.28571429],
       [0.71428571],
       [1.        ]])

In [218]:
np.abs(dum4).max()

7

In [219]:
dum4/np.max(np.abs(dum4))

array([[0.57142857],
       [0.14285714],
       [0.28571429],
       [0.85714286],
       [0.85714286],
       [0.85714286],
       [0.57142857],
       [0.28571429],
       [0.71428571],
       [1.        ]])

1. Pilih satu scaler  
2. Lakukan scaling pada X_train (paska imputasi) kolom numerik  
3. Print dataframe before and after  
4. Print .describe() before and after  

In [220]:
X_train_imp.head()

,Pclass,Sex,Age,SibSp,Fare,Embarked
0,3.0,male,32.0,0.0,8.0500,S
1,1.0,female,31.0,0.0,164.8667,S
2,3.0,female,22.2,1.0,23.4500,S
3,3.0,male,21.0,0.0,8.4333,S
4,1.0,male,30.0,0.0,45.5000,S


In [221]:
X_train_imp_num = X_train_imp.select_dtypes('number')
X_train_imp_cat = X_train_imp.select_dtypes('object')

In [222]:
X_train_imp_num.head()

,Pclass,Age,SibSp,Fare
0,3.0,32.0,0.0,8.0500
1,1.0,31.0,0.0,164.8667
2,3.0,22.2,1.0,23.4500
3,3.0,21.0,0.0,8.4333
4,1.0,30.0,0.0,45.5000


In [223]:
X_train_imp_num_scaled = ss.fit_transform(X_train_imp_num[['Pclass', 'Age', 'SibSp', 'Fare']])
X_train_imp_num_scaled

array([[ 0.83946169,  0.1564232 , -0.47350156, -0.47346752],
       [-1.5619492 ,  0.08074015, -0.47350156,  2.45248078],
       [ 0.83946169, -0.58527066,  0.51603163, -0.1861282 ],
       ...,
       [ 0.83946169,  0.45915538, -0.47350156, -0.48380801],
       [-0.36124376,  1.1403028 , -0.47350156, -0.0639157 ],
       [ 0.83946169,  0.61052147, -0.47350156, -0.47634465]])

In [226]:
df_X_train_imp_num_scaled = pd.DataFrame(X_train_imp_num_scaled, columns=['Pclass', 'Age', 'SibSp', 'Fare'])
df_X_train_imp_num_scaled

,Pclass,Age,SibSp,Fare
0,0.839462,0.156423,-0.473502,-0.473468
1,-1.561949,0.080740,-0.473502,2.452481
2,0.839462,-0.585271,0.516032,-0.186128
3,0.839462,-0.676090,-0.473502,-0.466316
4,-1.561949,0.005057,-0.473502,0.225289
...,...,...,...,...
1042,-1.561949,2.124182,-0.473502,2.239697
1043,-1.561949,2.275548,0.516032,4.283491
1044,0.839462,0.459155,-0.473502,-0.483808
1045,-0.361244,1.140303,-0.473502,-0.063916


In [227]:
print('--- Before Scaling ---')
print(X_train_imp_num.head())
print('--- After Scaling ---')
print(df_X_train_imp_num_scaled.head())

--- Before Scaling ---
   Pclass   Age  SibSp      Fare
0     3.0  32.0    0.0    8.0500
1     1.0  31.0    0.0  164.8667
2     3.0  22.2    1.0   23.4500
3     3.0  21.0    0.0    8.4333
4     1.0  30.0    0.0   45.5000
--- After Scaling ---
     Pclass       Age     SibSp      Fare
0  0.839462  0.156423 -0.473502 -0.473468
1 -1.561949  0.080740 -0.473502  2.452481
2  0.839462 -0.585271  0.516032 -0.186128
3  0.839462 -0.676090 -0.473502 -0.466316
4 -1.561949  0.005057 -0.473502  0.225289


In [228]:
print('--- Before Scaling ---')
print(X_train_imp_num.describe())
print('--- After Scaling ---')
print(df_X_train_imp_num_scaled.describe())

--- Before Scaling ---
            Pclass          Age       SibSp         Fare
count  1047.000000  1047.000000  1047.00000  1047.000000
mean      2.300860    29.933181     0.47851    33.425573
std       0.833242    13.219313     1.01106    53.620784
min       1.000000     0.170000     0.00000     0.000000
25%       2.000000    22.000000     0.00000     7.895800
50%       3.000000    28.000000     0.00000    13.858300
75%       3.000000    38.000000     1.00000    30.695800
max       3.000000    80.000000     8.00000   512.329200
--- After Scaling ---
             Pclass           Age         SibSp          Fare
count  1.047000e+03  1.047000e+03  1.047000e+03  1.047000e+03
mean  -7.804433e-17 -1.883244e-16 -2.035939e-17 -7.295448e-17
std    1.000478e+00  1.000478e+00  1.000478e+00  1.000478e+00
min   -1.561949e+00 -2.252568e+00 -4.735016e-01 -6.236676e-01
25%   -3.612438e-01 -6.004073e-01 -4.735016e-01 -4.763446e-01
50%    8.394617e-01 -1.463090e-01 -4.735016e-01 -3.650940e-01
75%    8